In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-24 23:31:26,0.26855,0.0,0.0,0.0,2022-06-24,23:31:26,23
1,2022-06-25 00:31:26,0.26855,0.0,0.0,0.0,2022-06-25,00:31:26,0
2,2022-06-25 01:31:26,0.26855,0.0,0.0,0.0,2022-06-25,01:31:26,1
3,2022-06-25 01:32:26,0.26855,0.0,0.0,0.0,2022-06-25,01:32:26,1
4,2022-06-25 01:33:26,0.31738,0.0,0.0,0.0,2022-06-25,01:33:26,1


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
11248,2022-07-03 14:52:46,2022-07-03,14:52:46,14,14.35547,1.71871,24.672890,0.024673
11249,2022-07-03 14:53:46,2022-07-03,14:53:46,14,14.30664,1.65995,23.748307,0.023748
11250,2022-07-03 14:54:46,2022-07-03,14:54:46,14,14.25781,1.54596,22.042004,0.022042
11251,2022-07-03 14:55:47,2022-07-03,14:55:47,14,14.13574,1.38094,19.520609,0.019521
11252,2022-07-03 14:56:47,2022-07-03,14:56:47,14,14.01367,1.21121,16.973497,0.016973


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-24T00:00:00.000000000', '2022-06-25T00:00:00.000000000',
       '2022-06-26T00:00:00.000000000', '2022-06-27T00:00:00.000000000',
       '2022-06-28T00:00:00.000000000', '2022-06-29T00:00:00.000000000',
       '2022-06-30T00:00:00.000000000', '2022-07-01T00:00:00.000000000',
       '2022-07-02T00:00:00.000000000', '2022-07-03T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
length_today_hourly_values = len(today_hourly_values)
length_today_hourly_values
# today_hourly_values

15

In [7]:
filter_daily_values = df[(df['Date'] > '2022-06-24') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
1,2022-06-25,0,0.0
2,2022-06-25,1,0.0
3,2022-06-25,1,0.0
4,2022-06-25,1,0.0
5,2022-06-25,1,0.0
...,...,...,...
10353,2022-07-02,20,0.0
10354,2022-07-02,21,0.0
10355,2022-07-02,22,0.0
10356,2022-07-02,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values

,Date,Hour,Power (KW)
0,2022-06-25,0,0.000000
1,2022-06-25,1,0.000000
2,2022-06-25,2,0.000000
3,2022-06-25,3,0.000000
4,2022-06-25,4,0.000000
...,...,...,...
187,2022-07-02,19,0.253908
188,2022-07-02,20,0.034134
189,2022-07-02,21,0.000000
190,2022-07-02,22,0.000000


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data.drop(['Date', 'Time', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
weather_data.tail(14)

,SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
202,478.4,16.4,67,56
203,461.1,16.9,65,66
204,458.9,18.9,55,70
205,466.6,20.1,51,70
206,559.1,19.9,48,54
207,612.3,19.5,47,41
208,632.2,19.3,46,29
209,546.1,19.0,44,32
210,443.9,18.7,44,35
211,327.2,18.0,46,38


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(25)

,Power (KW),SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
191,0.0,0.0,0.0,0,0
192,NaN,0.0,0.0,0,0
193,NaN,0.0,0.0,0,0
194,NaN,0.0,0.0,0,0
195,NaN,0.0,0.0,0,0
196,NaN,0.0,0.0,0,0
197,NaN,36.2,9.4,100,46
198,NaN,129.5,10.2,100,70
199,NaN,218.2,11.6,97,70
200,NaN,234.6,13.2,87,85


In [12]:
count_total_rows = len(df1) - 24

In [13]:
independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]

In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
rfr = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfr.fit(independent_columns, dependent_column)

RandomForestRegressor(random_state=0)

#### Is model well fit or not?

In [16]:
r_sq = rfr.score(independent_columns, dependent_column)
r_sq

0.9454040222610138

In [17]:
forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(24)

In [18]:
return_array = list(rfr.predict(forcasted_data))

In [19]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [20]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [21]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [22]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [23]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-07-03,0,0.000000
1,2022-07-03,1,0.000000
2,2022-07-03,2,0.000000
3,2022-07-03,3,0.000000
4,2022-07-03,4,0.000000
5,2022-07-03,5,0.018157
6,2022-07-03,6,0.130286
7,2022-07-03,7,0.113815
8,2022-07-03,8,0.826748
9,2022-07-03,9,0.920913


In [24]:
data_dataframe['Power (KW)'].sum()

15.398936217846233

## Evaluating the model

#### MSE = mean_squared_error(actual value, predicted value)

In [25]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_sq_error

0.11736936219973233

In [26]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

0.34259212220909624

In [27]:
mean_ab_error = metrics.mean_absolute_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_ab_error

0.1819936570157039

#### R squared

In [28]:
r_squared = metrics.r2_score(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
r_squared

0.746058263769489

#### Yesterday results

In [31]:
filter_last_day_values = df[df['Date'] == unique_date[-2]][['Date', 'Hour', 'Power (KW)']]
last_day_hourly_values = filter_last_day_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

filter_yes_values = df[(df['Date'] >= '2022-06-25') & (df['Date'] <= unique_date[-3])][['Date', 'Hour', 'Power (KW)']]
yes_hourly_values = filter_yes_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data1 = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_unique_date = weather_data1['Date'].unique()
filter_weather_yes_values = weather_data1[
                                         (weather_data1['Date'] >= '2022-06-25') & 
                                         (weather_data1['Date'] <= weather_unique_date[-3])][['SolarIrradiance (W/m2)',
                                                                                             'Temp (°C)', 'Hum (%)',
                                                                                             'CloudCover (%)']]
yes_df1 = pd.concat([yes_hourly_values, filter_weather_yes_values], axis = 1)
yes_df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
yes_df1.loc[yes_df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
yes_count_total_rows = len(yes_df1)
yes_independent_columns = yes_df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:yes_count_total_rows]
yes_dependent_column = yes_df1['Power (KW)'][0:yes_count_total_rows]
rfr_yes = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfr_yes.fit(yes_independent_columns, yes_dependent_column)
forcasted_yes_values = weather_data1[(weather_data1['Date'] == weather_unique_date[-2])][['SolarIrradiance (W/m2)',
                                                                                        'Temp (°C)', 'Hum (%)',
                                                                                        'CloudCover (%)']]
forcasted_yes_values.loc[forcasted_yes_values['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
rfr_yes_return_array = rfr_yes.predict(forcasted_yes_values)
rfr_yes_predicted_data = pd.DataFrame(rfr_yes_return_array, columns = ['Power (KW)'])
rfr_yes_pe = rfr_yes_predicted_data['Power (KW)'].sum()
rfr_yes_mse = metrics.mean_squared_error(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_rmse = np.sqrt(rfr_yes_mse)
rfr_yes_mae = metrics.mean_absolute_error(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_rs = metrics.r2_score(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_rs

-0.15547195597848718